# Description

It runs the GWAS imputation step from the script in: https://github.com/hakyimlab/summary-gwas-imputation

# Modules

In [1]:
import os
from glob import glob
from pathlib import Path

from utils import read_log_file_and_check_line_exists
import conf

# Settings

Apparently, there is no easy way to get the parent directory of
a notebook in Jupyter, so here I get that information either from
the parameter sent by `nbs/run_nbs.sh` (if called from command-line) or
from `os.getcwd()` (if called from browser).

In [2]:
PHENOPLIER_NOTEBOOK_FILEPATH = None
PHENOPLIER_NOTEBOOK_DIR = os.getcwd()

In [3]:
# Parameters
PHENOPLIER_NOTEBOOK_FILEPATH = (
    "projects/asthma-copd/nbs/10_gwas_harmonization/05-run_imputation.ipynb"
)

In [4]:
if PHENOPLIER_NOTEBOOK_FILEPATH is not None:
    PHENOPLIER_NOTEBOOK_DIR = str(Path(PHENOPLIER_NOTEBOOK_FILEPATH).parent)

display(PHENOPLIER_NOTEBOOK_DIR)

'projects/asthma-copd/nbs/10_gwas_harmonization'

In [5]:
OUTPUT_DIR = conf.PROJECTS["ASTHMA_COPD"]["RESULTS_DIR"] / "imputed_gwas"
display(OUTPUT_DIR)

OUTPUT_DIR_STR = str(OUTPUT_DIR)
display(OUTPUT_DIR_STR)

PosixPath('/opt/data/projects/asthma-copd/results/imputed_gwas')

'/opt/data/projects/asthma-copd/results/imputed_gwas'

# Run

In [6]:
%%bash -s "$PHENOPLIER_NOTEBOOK_DIR" "$OUTPUT_DIR_STR"
set -euo pipefail
# IFS=$'\n\t'

# read the notebook directory parameter and remove $1
export PHENOPLIER_NOTEBOOK_DIR="${PHENOPLIER_CODE_DIR}/$1"
shift

# read output dir
export OUTPUT_DIR="$1"
shift

run_job () {
    # run_job is a standard function name that performs a particular job
    # depending on the context. It will be called by GNU Parallel below.
    #
    # The implementation here runs the GLS model of PhenoPLIER on a trait.

    # read trait information
    # the first parameter to this function is a string with values separated by
    # commas (,). So here I split those into different variables.
    IFS=',' read -r pheno_id file sample_size n_cases chromosome batch_id <<< "$1"

    INPUT_FILENAME="${file%.*}"

    N_BATCHES=10

    # IMPUTED_GWAS_DIR="${PHENOPLIER_PROJECTS_ASTHMA_COPD_RESULTS_DIR}/imputed_gwas"
    # INPUTED_GWAS_FILE_PATTERN="${INPUT_FILENAME}-harmonized"

    # get input GWAS file, there should be a single file
    # here I make sure that there are no other files in the folder that
    # match this phenotype/trait filename prefix
    GWAS_DIR="${PHENOPLIER_PROJECTS_ASTHMA_COPD_RESULTS_DIR}/harmonized_gwas"
    N_GWAS_FILES=$(ls ${GWAS_DIR}/${INPUT_FILENAME}*.txt | wc -l)
    if [ "${N_GWAS_FILES}" != "1" ]; then
        echo "ERROR: found ${N_GWAS_FILES} GWAS files instead of one"
        exit 1
    fi
    INPUT_GWAS_FILEPATH=$(ls ${GWAS_DIR}/${INPUT_FILENAME}*.txt)

    mkdir -p "${OUTPUT_DIR}"

    # make sure we are not also parallelizing within numpy, etc
    export NUMBA_NUM_THREADS=1
    export MKL_NUM_THREADS=1
    export OPEN_BLAS_NUM_THREADS=1
    export NUMEXPR_NUM_THREADS=1
    export OMP_NUM_THREADS=1

    echo "Running for $pheno_id, $chromosome, $batch_id"
    echo "Saving results in ${OUTPUT_DIR}"

    bash "${PHENOPLIER_CODE_DIR}/scripts/gwas_impute.sh" \
        --input-gwas-file "${INPUT_GWAS_FILEPATH}" \
        --chromosome "${chromosome}" \
        --n-batches "${N_BATCHES}" \
        --batch-id "${batch_id}" \
        --output-dir "${OUTPUT_DIR}"

    # print errors here in the notebook
    # first, look for the log file for this trait
    pattern="${OUTPUT_DIR}/${INPUT_FILENAME}*-chr${chromosome}-batch${batch_id}_${N_BATCHES}*.log"

    N_LOG_FILES=$(ls ${pattern} | wc -l)
    if [ "${N_LOG_FILES}" != "1" ]; then
        echo "ERROR: found ${N_LOG_FILES} log files instead of one"
        exit 1
    fi
    LOG_FILE=$(ls ${pattern})

    cat "${LOG_FILE}" | grep -iE "warning|error"

    echo
}

# export function so GNU Parallel can see it
export -f run_job

# generate a list of run_job calls for GNU Parallel
# here I read a file with information about traits (one trait per line)
while IFS= read -r line; do
    for chromosome in {1..22}; do
        for batch_id in {0..9}; do
            echo run_job "${line},${chromosome},${batch_id}"
            # echo run_job "${line},1,2"
            # break
        done
        # break
    done
done < <(tail -n "+2" "${PHENOPLIER_PROJECTS_ASTHMA_COPD_TRAITS_INFO_FILE}") |
    parallel -k --lb --halt 2 -j${PHENOPLIER_GENERAL_N_JOBS}

Running for asthma_only, 1, 0
Saving results in /opt/data/projects/asthma-copd/results/imputed_gwas


+ /opt/data/software/conda_envs/summary_gwas_imputation/bin/python /opt/data/software/summary-gwas-imputation/src/gwas_summary_imputation.py -by_region_file /opt/data/data/eur_ld.bed.gz -gwas_file /opt/data/projects/asthma-copd/results/harmonized_gwas/GWAS_Asthma_only_GLM_SNPs_info0.7-harmonized.txt -parquet_genotype /opt/data/data/phenomexcan/ld_blocks/reference_panel_1000G/chr1.variants.parquet -parquet_genotype_metadata /opt/data/data/phenomexcan/ld_blocks/reference_panel_1000G/variant_metadata.parquet -window 100000 -parsimony 7 -chromosome 1 -regularization 0.1 -frequency_filter 0.01 -sub_batches 10 -sub_batch 0 --standardise_dosages -output /opt/data/projects/asthma-copd/results/imputed_gwas/GWAS_Asthma_only_GLM_SNPs_info0.7-harmonized-imputed-chr1-batch0_10.txt


/opt/data/software/conda_envs/summary_gwas_imputation/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.



+ set +x


Running for asthma_only, 1, 1
Saving results in /opt/data/projects/asthma-copd/results/imputed_gwas
/opt/data/software/conda_envs/summary_gwas_imputation/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
INFO - Error for region (1,34334157.0,37083582.0): LinAlgError('SVD did not converge')



...

+ set +x


Running for aco, 20, 8
Saving results in /opt/data/projects/asthma-copd/results/imputed_gwas
/opt/data/software/conda_envs/summary_gwas_imputation/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.

Running for aco, 20, 9
Saving results in /opt/data/projects/asthma-copd/results/imputed_gwas
/opt/data/software/conda_envs/summary_gwas_imputation/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.

Running for aco, 21, 0
Saving results in /opt/data/projects/asthma-copd/results/imputed_gwas
/opt/data/software/conda_envs/summary_gwas_imputation/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.

Running for aco, 21, 1
Saving results in /opt/data/projects/asthma-copd/results/imputed_gwas
/opt/data/software/conda_envs/summary_gwas_imputation/lib/

+ /opt/data/software/conda_envs/summary_gwas_imputation/bin/python /opt/data/software/summary-gwas-imputation/src/gwas_summary_imputation.py -by_region_file /opt/data/data/eur_ld.bed.gz -gwas_file /opt/data/projects/asthma-copd/results/harmonized_gwas/GWAS_ACO_GLM_SNPs_info0.7-harmonized.txt -parquet_genotype /opt/data/data/phenomexcan/ld_blocks/reference_panel_1000G/chr20.variants.parquet -parquet_genotype_metadata /opt/data/data/phenomexcan/ld_blocks/reference_panel_1000G/variant_metadata.parquet -window 100000 -parsimony 7 -chromosome 20 -regularization 0.1 -frequency_filter 0.01 -sub_batches 10 -sub_batch 8 --standardise_dosages -output /opt/data/projects/asthma-copd/results/imputed_gwas/GWAS_ACO_GLM_SNPs_info0.7-harmonized-imputed-chr20-batch8_10.txt
+ set +x
+ /opt/data/software/conda_envs/summary_gwas_imputation/bin/python /opt/data/software/summary-gwas-imputation/src/gwas_summary_imputation.py -by_region_file /opt/data/data/eur_ld.bed.gz -gwas_file /opt/data/projects/asthma-co

Running for aco, 21, 5
Saving results in /opt/data/projects/asthma-copd/results/imputed_gwas
/opt/data/software/conda_envs/summary_gwas_imputation/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.

Running for aco, 21, 6
Saving results in /opt/data/projects/asthma-copd/results/imputed_gwas
/opt/data/software/conda_envs/summary_gwas_imputation/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.

Running for aco, 21, 7
Saving results in /opt/data/projects/asthma-copd/results/imputed_gwas
/opt/data/software/conda_envs/summary_gwas_imputation/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.

Running for aco, 21, 8
Saving results in /opt/data/projects/asthma-copd/results/imputed_gwas
/opt/data/software/conda_envs/summary_gwas_imputation/lib/

+ /opt/data/software/conda_envs/summary_gwas_imputation/bin/python /opt/data/software/summary-gwas-imputation/src/gwas_summary_imputation.py -by_region_file /opt/data/data/eur_ld.bed.gz -gwas_file /opt/data/projects/asthma-copd/results/harmonized_gwas/GWAS_ACO_GLM_SNPs_info0.7-harmonized.txt -parquet_genotype /opt/data/data/phenomexcan/ld_blocks/reference_panel_1000G/chr21.variants.parquet -parquet_genotype_metadata /opt/data/data/phenomexcan/ld_blocks/reference_panel_1000G/variant_metadata.parquet -window 100000 -parsimony 7 -chromosome 21 -regularization 0.1 -frequency_filter 0.01 -sub_batches 10 -sub_batch 5 --standardise_dosages -output /opt/data/projects/asthma-copd/results/imputed_gwas/GWAS_ACO_GLM_SNPs_info0.7-harmonized-imputed-chr21-batch5_10.txt
+ set +x
+ /opt/data/software/conda_envs/summary_gwas_imputation/bin/python /opt/data/software/summary-gwas-imputation/src/gwas_summary_imputation.py -by_region_file /opt/data/data/eur_ld.bed.gz -gwas_file /opt/data/projects/asthma-co

# Perform some checks in output and log files

In [7]:
assert OUTPUT_DIR.exists()

In [8]:
log_files = OUTPUT_DIR.glob("*.log")

In [9]:
for f in log_files:
    read_log_file_and_check_line_exists(
        f,
        [
            "INFO - Finished in",
        ],
    )